In [4]:
import numpy as np
import pandas as pd

## Creating Pandas Dataframe

In [1]:
df_info = pd.read_excel('./dataset/2/movie_info.xlsx')
df_review = pd.read_excel('./dataset/2/movie_reviews.xls')
df_user = pd.read_excel('./dataset/2/userdata.xls')
print("Shapes of df:")
print(f"Movie Info: {df_info.shape}")
print(f"User Data: {df_user.shape}")
print(f"Movie Review: {df_review.shape}")

NameError: name 'pd' is not defined

## Creating Database

In [46]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="temp",
    passwd="password"
    
)

mycursor = mydb.cursor()
mycursor.execute("drop database RandomX;")
mycursor.execute("create database RandomX;")
mycursor.close()


True

## Creating Tables

In [47]:
mydb = mysql.connector.connect(
    host="localhost",
    user="temp",
    passwd="password",
    database ="RandomX"
    
)

mycursor = mydb.cursor()
create_table = """

CREATE TABLE if not exists `user_details` (
  `user_name` varchar(50),
  `gender` varchar(50),
  `age` int,
  PRIMARY KEY (`user_name`)
);

CREATE TABLE if not exists `movie_details` (
  `movie_id` varchar(80),
  `movie_title` varchar(100),
  `audience_rating` float,
  PRIMARY KEY (`movie_id`)
);

CREATE TABLE if not exists `movie_reviews` (
  `review_id` int not null auto_increment,
  `movie_id` varchar(80),
  `user_name` varchar(50),
  `movie_rev` varchar(1000),
  PRIMARY KEY (`review_id`),
  FOREIGN KEY (movie_id) REFERENCES movie_details(movie_id),
  FOREIGN KEY (user_name) REFERENCES user_details(user_name)

);
"""
mycursor.execute(create_table)

mycursor.close()


True

## Inserting Data:

In [48]:
mydb = mysql.connector.connect(
    host="localhost",
    user="temp",
    passwd="password",
    database ="RandomX"
    
)



def list_creator (df):
    column_names = df.columns
    
    temp_list=[]
    for i in range(len(df)):
        
        temp_query= []
        for j in range(len(column_names)):
            item=column_names[j]
            temp_query.append(str(df[item][i]))

    
        temp_list.append(tuple(temp_query))
    return temp_list


user_list = list_creator(df_user)
movie_list = list_creator(df_info)
review_list = list_creator(df_review)

mycursor = mydb.cursor()

user_query="INSERT INTO user_details (user_name,gender,age) VALUES (%s,%s,%s)"
movie_query = "INSERT INTO movie_details (movie_id,movie_title,audience_rating) VALUES (%s,%s,%s)"
review_query = "INSERT INTO movie_reviews (movie_id,user_name,movie_rev) VALUES (%s,%s,%s)"

mycursor.executemany(user_query,user_list)
mycursor.executemany(movie_query,movie_list)
mycursor.executemany(review_query,review_list)

mydb.commit()
mycursor.close()

True

## Verifying Whether all data is inserted

In [49]:
mydb = mysql.connector.connect(
    host="localhost",
    user="temp",
    passwd="password",
    database ="RandomX"
    
)

mycursor = mydb.cursor()

mycursor.execute('select COUNT(*) from user_details;')
for line in mycursor:
    print("Number of Users:",line)

mycursor.execute('select COUNT(*) from movie_details;')
for line in mycursor:
    print("Number of Movies:",line)

mycursor.execute('select COUNT(*) from movie_reviews;')
for line in mycursor:
    print("Number of Reviews",line)


Number of Users: (20,)
Number of Movies: (100,)
Number of Reviews (500,)


## 3. Updating Sentiment of the Review

In [50]:
'''
A dictionary is created.
where,
        key =word
        value = 1 if word is positive
               -1 if word is negative

'''

positive_f = open('./dataset/2/positive_words.txt','r')
negative_f = open('./dataset/2/negative_words.txt','r')


word_dictionary ={}
for line in positive_f:
    temp=line.rstrip('\n')
    word_dictionary[temp]=1

for line in negative_f:
    temp=line.rstrip('\n')
    word_dictionary[temp]= -1




In [51]:
'''
To find the sentiment of each review. We traverse through each words of the review.

We initialize the sentiment score =0

Then if the words is in the dictionary, the respective value is added to the sentiment score

if the sentiment score is positive, it means more positive words and the sentence is positive. Similary if it is negative, the sentence would be negative and would be neutral if the score is zero.

'''



def find_sentiment(review_list):
    sentiment_array=[0]*len(review_list)

    for i in range(len(review_list)):
        item = review_list[i]
        temp_list =item[-1].split()
        sentiment_score=0
        for word in temp_list:
            try:
                if word_dictionary[word]:
                    sentiment_score+=word_dictionary[word]
            except:
                continue
        
        if sentiment_score>0:
            sentiment_array[i]='positive'
        
        elif sentiment_score<0:
            sentiment_array[i]='negative'
        else:
            sentiment_array[i]='neutral'
    return sentiment_array

review_list = list_creator(df_review)
sentiment_array=find_sentiment(review_list)

In [55]:
mydb = mysql.connector.connect(
    host="localhost",
    user="temp",
    passwd="password",
    database ="RandomX"
    
)

mycursor = mydb.cursor()

alter_table = """

ALTER TABLE movie_reviews
ADD COLUMN sentiment VARCHAR(20) AFTER movie_rev;
"""

mycursor.execute(alter_table)


update_query = """
UPDATE movie_reviews SET
sentiment=%s WHERE
review_id=%s
""" 

sentiment = [(sentiment_array[i],i+1) for i in range(len(review_list))]
mycursor.executemany(update_query,sentiment)
mydb.commit()
mycursor.close()

ProgrammingError: 1060 (42S21): Duplicate column name 'sentiment'

## 4. Show the top 5 movie names with a rating strictly greater than 3.5, and the count of ‘positive’ sentiment is greater than equal to 2 for that movie.

In [5]:

'''
First movie_review table and movie_details table is joined on conditions both of their movie_ids are equal, audience rating is greater 3.5 and sentiment is positive.


Now, the count of each movies is taken which and new table is created.

From the new table, the movies with the count of positive review greater than 2 is filtered out and arranged first in the decreasing order of audience rating and then the decreasing order of number of positive reviews.


'''
import mysql.connector



mydb = mysql.connector.connect(
    host="localhost",
    user="temp",
    passwd="password",
    database ="RandomX"
    
)

mycursor = mydb.cursor()


query= """


select B.movie_title,B.audience_rating,
B.Total_Positive
from
(select A.movie_title,A.audience_rating,
COUNT(A.movie_title) Total_Positive
from
(select movie_details.movie_id,movie_details.movie_title,
movie_details.audience_rating,movie_reviews.sentiment
from movie_details inner join movie_reviews
on movie_details.movie_id=movie_reviews.movie_id 
and movie_reviews.sentiment='positive' 
and movie_details.audience_rating>3.5) as A
group by A.movie_title,A.audience_rating) as B
where B.Total_Positive>=2 order by B.audience_rating desc,
B.Total_Positive desc limit 5;
"""

mycursor.execute(query)
result=[]
for line in mycursor:
    result.append(line)

df =pd.DataFrame(result,columns=['Movie_Name','Audience_Rating','Count of Positive Sentiment'])

df

,Movie_Name,Audience_Rating,Count of Positive Sentiment
0,The Lord of the Rings: The Fellowship of the Ring,4.75,3
1,Room,4.65,5
2,The Man Who Shot Liberty Valance,4.60,3
3,Run Lola Run,4.50,2
4,Star Trek Into Darkness,4.45,5
